In [17]:
os.environ["mongo_server"] = "HIDDEN"
os.environ["gmaps_api_key"] ="HIDDEN"

In [20]:
import datetime
import pandas as pd
import numpy as np
import os
import googlemaps

from pymongo import MongoClient
from math import radians, cos, sin, asin, sqrt
from bson.son import SON


MONGO_URL = os.environ["mongo_server"] # Dirección mlab para la base de datos
GMAPS_APIKEY = os.environ["gmaps_api_key"]

client = MongoClient(MONGO_URL)
gmaps = googlemaps.Client(key=GMAPS_APIKEY)

db = client.datalab
locations = db.locations

def get_near_documents(longitude,latitude):
    max_distance = 0
    objects = 0
    while objects <= 10:
        max_distance = max_distance + 1000
        query = {'geopoint': {
        '$near': SON([('$geometry', SON([
            ('type', 'Point'),
            ('coordinates', [longitude, latitude])])),
            ('$maxDistance', max_distance)])}}
        response =list(locations.find(query))
        objects = len(response)
    return response


def list_geopoint_extract(place_list):
    geopoint = place_list["geopoint"]
    longitude = geopoint["coordinates"][0]
    latitude = geopoint["coordinates"][1]
    return longitude, latitude


def haversine(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    m = 6367000 * c
    return m


def coords_stringify(longitude,latitude):
    output = str(latitude) + ","+ str(longitude)
    return output


def get_directions(longitude1, latitude1, longitude2, latitude2, route_type):
    try:
        dir1string = coords_stringify(longitude1, latitude1)
        dir2string= coords_stringify(longitude2, latitude2)
        directions_result = gmaps.directions(dir1string,dir2string, mode=route_type, departure_time= datetime.datetime.now())
        distance = directions_result[0]["legs"][0]["distance"]["value"]
        route = directions_result[0]["legs"][0]["steps"]
    except:
        distance = np.nan
        route = np.nan
    return distance, route

def route_distance_info(row,longitude1, latitude1, route_type):
    longitude2, latitude2 = list_geopoint_extract(row)
    if route_type == "euclidean":
        distance = haversine(longitude1, latitude1, longitude2, latitude2)
        route = np.nan
    elif route_type == "walking":
        distance, route = get_directions(longitude1, latitude1, longitude2, latitude2, "walking")
    else:
        distance, route = get_directions(longitude1, latitude1, longitude2, latitude2, "driving")
    return distance, route


def directions_row(row, longitude1, latitude1, route_type):
    row["distance"], row["route"] = route_distance_info(row,longitude1, latitude1, route_type)
    return row

def find_nearest_top(longitude, latitude, mode, head):
    near_places = get_near_documents(longitude, latitude)
    dataframe_places = pd.DataFrame(near_places)
    dataframe_distances= dataframe_places.apply(directions_row,  axis=1, args =(longitude, latitude, mode))
    if mode is not "euclidean":
        dataframe_distances = dataframe_distances.dropna()
    df_distances_top = dataframe_distances.head(head)
    return df_distances_top

def nearest_response(longitude, latitude, mode, head):
    response_df = find_nearest_top(longitude, latitude, mode,head)
    response_dict = response_df.to_dict(orient="records")
    return(json.dumps(response_dict).replace('": NaN','": "NaN"'))

In [21]:
a =find_nearest_top(-104.430884, 21.035612, "euclidean",3)

      _id  altitud  capacidad                         direccion  \
0    24.0   1000.0       60.0                           ZOATLAN   
1    25.0   1000.0       20.0                           ZOATLAN   
2    26.0   1000.0       40.0                        LA CIENEGA   
3   316.0   1140.0      120.0  AGUSTIN MELGAR ESQ. JUAN ESCUTIA   
4   319.0   1140.0      108.0                 EULOGIO PARRA S/N   
5    22.0   1000.0      150.0                       MORELOS S/N   
6    23.0   1000.0      250.0                       MORELOS S/N   
7    18.0   1000.0      200.0                  OAXACA E HIDALGO   
8   314.0   1140.0      108.0              HIDALGO NO. 381 PTE.   
9    15.0   1000.0      250.0        20 DE NOVIEMBRE Y ZARAGOZA   
10   21.0   1000.0      200.0                       HIDALGO S/N   
11   20.0   1000.0      400.0                  20 DE NOVIEMBRE    
12   14.0   1000.0       80.0                  20 DE NOVIEMBRE    
13  313.0   1140.0      136.0              HIDALGO NO. 329 PTE

In [35]:
a = nearest_response(-104.430884, 21.035612, "euclidean",3)

In [36]:
a

'[{"altitud": 1000.0, "distance": 495.8561047673046, "latitud": "21\\u00aa01\'52.14\\"", "responsable": "LETICIA LOPEZ JIMENMEZ", "geopoint": {"type": "Point", "coordinates": [-104.43092222222221, 21.03115]}, "refugio": "ESCUELA PRIMARIA AMADO NERVO", "longitud": "104\\u00aa25\'51.32", "capacidad": 60.0, "telefono": "311-2142276", "uso": "EDUCACION", "direccion": "ZOATLAN", "servicios": "AGUA, SANITARIOS Y LUZ ELECTRICA", "_id": 24.0, "route": "NaN", "municipio": "AHUACATLAN"}, {"altitud": 1000.0, "distance": 495.8561047673046, "latitud": "21\\u00aa01\'52.14\\"", "responsable": "GREGORIA SOLIS CIBRIAN", "geopoint": {"type": "Point", "coordinates": [-104.43092222222221, 21.03115]}, "refugio": "ESC. TELESECUNDARIA FRANCISCO VILLA", "longitud": "104\\u00aa25\'51.32", "capacidad": 20.0, "telefono": "311-2142276", "uso": "EDUCACION", "direccion": "ZOATLAN", "servicios": "AGUA, SANITARIOS Y LUZ ELECTRICA", "_id": 25.0, "route": "NaN", "municipio": "AHUACATLAN"}, {"altitud": 1000.0, "distance

In [28]:
import json

In [24]:
import datetime
import os
import pandas as pd
import numpy as np
import googlemaps

from pymongo import MongoClient
from math import radians, cos, sin, asin, sqrt
from bson.son import SON


MONGO_URL = os.environ["mongo_server"] # Dirección mlab para la base de datos
GMAPS_APIKEY = os.environ["gmaps_api_key"]

client = MongoClient(MONGO_URL)
gmaps = googlemaps.Client(key=GMAPS_APIKEY)

db = client.datalab
locations = db.locations

def get_near_documents(longitude,latitude):
    max_distance = 0
    objects = 0
    while objects <= 10:
        max_distance = max_distance + 1000
        query = {'geopoint': {
        '$near': SON([('$geometry', SON([
            ('type', 'Point'),
            ('coordinates', [longitude, latitude])])),
            ('$maxDistance', max_distance)])}}
        response =list(locations.find(query))
        objects = len(response)
    return response


def list_geopoint_extract(place_list):
    geopoint = place_list["geopoint"]
    longitude = geopoint["coordinates"][0]
    latitude = geopoint["coordinates"][1]
    return longitude, latitude


def haversine(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    m = 6367000 * c
    return m


def coords_stringify(longitude,latitude):
    output = str(latitude) + ","+ str(longitude)
    return output


def get_directions(longitude1, latitude1, longitude2, latitude2, route_type):
    try:
        dir1string = coords_stringify(longitude1, latitude1)
        dir2string= coords_stringify(longitude2, latitude2)
        directions_result = gmaps.directions(dir1string,dir2string, mode=route_type, departure_time= datetime.datetime.now())
        distance = directions_result[0]["legs"][0]["distance"]["value"]
        route = directions_result[0]["legs"][0]["steps"]
    except:
        distance = np.nan
        route = np.nan
    return distance, route

def route_distance_info(row,longitude1, latitude1, route_type):
    longitude2, latitude2 = list_geopoint_extract(row)
    if route_type == "euclidean":
        distance = haversine(longitude1, latitude1, longitude2, latitude2)
        route = np.nan
    elif route_type == "walking":
        distance, route = get_directions(longitude1, latitude1, longitude2, latitude2, "walking")
    else:
        distance, route = get_directions(longitude1, latitude1, longitude2, latitude2, "driving")
    return distance, route


def directions_row(row, longitude1, latitude1, route_type):
    row["distance"], row["route"] = route_distance_info(row,longitude1, latitude1, route_type)
    return row


def find_nearest_top(longitude, latitude, mode, head):
    near_places = get_near_documents(longitude, latitude)
    dataframe_places = pd.DataFrame(near_places)
    dataframe_distances= dataframe_places.apply(directions_row,  axis=1, args = (longitude, latitude, mode))
    if mode is not "euclidean":
        dataframe_distances = dataframe_distances.dropna()
    df_distances_top = dataframe_distances.head(head)
    return df_distances_top

def nearest_response(longitude, latitude, mode, head):
    response_df = find_nearest_top(longitude, latitude, mode, head)
    response_dict = response_df.to_dict(orient="records")
    return(response_dict)


In [1]:
import os

In [26]:
os.environ["mongo_server"] = "HIDDEN"
os.environ["gmaps_api_key"] ="HIDDEN"

In [25]:
nearest_response(-104.90595817565918,21.494302974868788, "euclidean", 3)

[{'_id': 270.0,
  'altitud': 930.0,
  'capacidad': 4000.0,
  'direccion': 'UNIVERSIDAD AUTONOMA DE NAYARIT',
  'distance': 1215.7549166292952,
  'geopoint': {'coordinates': [-104.8949027777778, 21.49057777777778],
   'type': 'Point'},
  'latitud': '21º29\'26.08"',
  'longitud': '104º53\'41.65"',
  'municipio': 'TEPIC',
  'refugio': 'U.A.N. EN SU TOTALIDAD',
  'responsable': 'RECTOR DE LA U.A.N.',
  'route': nan,
  'servicios': 'AGUA, SANITARIOS Y LUZ ELECTRICA',
  'telefono': '311-2118800',
  'uso': 'EDUCACION'},
 {'_id': 267.0,
  'altitud': 930.0,
  'capacidad': 300.0,
  'direccion': 'BOULEVARD TEPIC-XALISCO S/N',
  'distance': 1230.9234847038351,
  'geopoint': {'coordinates': [-104.89510833333334, 21.489744444444447],
   'type': 'Point'},
  'latitud': '21º29\'23.08"',
  'longitud': '104º53\'42.39"',
  'municipio': 'TEPIC',
  'refugio': 'COLEGIO CRISTÓBAL COLON',
  'responsable': 'ROBERTO ALDRETE PADILLA ',
  'route': nan,
  'servicios': 'AGUA, SANITARIOS Y LUZ ELECTRICA',
  'telefono